In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


In [213]:
def prepare_data(df,test_size,i):
    '''label = df[forecast_col].shift(-forecast_out) #creating new column called label with the last 5 rows are nan
    X = np.array(df[[forecast_col]]) #creating the feature array
    X = preprocessing.scale(X) #processing the feature array
    X_lately = X[-forecast_out:] #creating the column i want to use later in the predicting method
    X = X[:-forecast_out] # X that will contain the training and testing
    label.dropna(inplace=True) #dropping na values
    y = np.array(label)  # assigning Y'''
    X = df.loc[:, f'Open-Stock-{i}':f'Turnover-Stock-{i}']
    Y = df.loc[:,f'Close-Stock-{i}']
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=0) #cross validation

    response = [X_train,X_test , Y_train, Y_test]
    return response

In [173]:
df = pd.read_csv("new_train.csv")
#df = pd.concat([df,df1], sort = True)
#df.fillna(0)

In [214]:
df.fillna(0, inplace = True)
data = []


In [175]:
#test_data = pd.concat([df,df1], sort = False)
#test_data[-100:]

In [231]:
df1 = pd.read_csv("new_test.csv")
test_data = []

In [243]:
for i in range(1,6):
    test_data.append(df1[['Date', f'Open-Stock-{i}',f'High-Stock-{i}', f'Low-Stock-{i}', f'VWAP-Stock-{i}', f'Volume-Stock-{i}', f'Turnover-Stock-{i}' ]])
    test_data[i-1] = test_data[i-1].loc[:,f'Open-Stock-{i}': f'Turnover-Stock-{i}']
test_data[0].head()

,Open-Stock-1,High-Stock-1,Low-Stock-1,VWAP-Stock-1,Volume-Stock-1,Turnover-Stock-1
0,484.1,488.90,475.00,481.00,1829606,88003633160000
1,465.0,475.80,457.10,469.56,3156094,148196817355000
2,463.0,480.00,458.05,465.04,4308373,200357861965000
3,450.0,473.55,449.85,460.88,4490924,206978212685000
4,494.8,494.80,473.40,478.06,3267226,156192368225000


In [215]:
for i in range(1,6):
    data.append(df[['Date', f'Open-Stock-{i}',f'High-Stock-{i}', f'Low-Stock-{i}', f'VWAP-Stock-{i}', f'Volume-Stock-{i}', f'Turnover-Stock-{i}',  f'Close-Stock-{i}' ]])
    


In [216]:
#x = data[0].loc[:, 'Open-Stock-1':'Turnover-Stock-1']
#x.head()

In [217]:
test_size = []

In [256]:
for i in range(1,6):
    test_size.append(0.20)

In [257]:
X_train, X_test, Y_train, Y_test  = [0 for i in range(5)], [0 for i in range(5)], [0 for i in range(5)], [0 for i in range(5)]
learner = [1,1,1,1,1]

In [258]:
for i in range(5):
    X_train[i], X_test[i], Y_train[i], Y_test[i] =prepare_data(data[i],test_size[i],i+1)
    learner[i] = LinearRegression()
    learner[i].fit(X_train[i],Y_train[i])

In [259]:
response={}
for i in range(5):
    score=learner[i].score(X_test[i],Y_test[i])#testing the linear regression model
    #forecast= learner[i].predict(X_lately[i]) #set that will contain the forecasted data
    response[f'test_score-{i+1}']=score
    #response[f'forecast_set-{i+1}']=forecast

In [260]:
print(response)

{'test_score-1': 0.9990515042642574, 'test_score-2': 0.9989971745575823, 'test_score-3': 0.9935925019112262, 'test_score-4': 0.9912800032158363, 'test_score-5': 0.9886046306777515}


In [261]:
forecast = {}
for i in range(5):
    forecast[f'Close-Stock-{i+1}'] = learner[i].predict(test_data[i])

In [262]:
final = pd.DataFrame(forecast)

In [263]:
final.head()

,Close-Stock-1,Close-Stock-2,Close-Stock-3,Close-Stock-4,Close-Stock-5
0,479.667408,202.366006,3084.953802,999.938524,1294.683555
1,470.449775,194.575086,3045.467031,959.648997,1220.316296
2,470.607667,194.752952,3072.723835,959.585349,1242.834383
3,468.387449,198.866506,3110.561047,948.366794,1207.802591
4,476.247326,198.333185,3170.296682,964.744691,1240.161677


In [264]:
final['Date'] = df1['Date'].values

In [265]:
final.head()

,Close-Stock-1,Close-Stock-2,Close-Stock-3,Close-Stock-4,Close-Stock-5,Date
0,479.667408,202.366006,3084.953802,999.938524,1294.683555,2011-08-04
1,470.449775,194.575086,3045.467031,959.648997,1220.316296,2011-08-05
2,470.607667,194.752952,3072.723835,959.585349,1242.834383,2011-08-08
3,468.387449,198.866506,3110.561047,948.366794,1207.802591,2011-08-09
4,476.247326,198.333185,3170.296682,964.744691,1240.161677,2011-08-10


In [266]:
final = final[['Date', 'Close-Stock-1', 'Close-Stock-2','Close-Stock-3','Close-Stock-4', 'Close-Stock-5']]

In [255]:
final.to_csv('output.csv', index = False)